In [ ]:
# -*- coding: utf-8
# Repórter Brasil (https://ruralometro2022.reporterbrasil.org.br/)
# Reinaldo Chaves (@paidatocandeira)
# Scripts para somar, sem repetições as doações dos deputados em 2018 - relacionadas às listas de 
# pessoas/empresas em infrações trabalhistas

In [150]:
import pandas as pd

### Doacoes gerais diretas

In [151]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
doacoes_2018 = pd.read_csv("resultados/doacoes/doacoes_geral_politicos_2018.csv", **kwargs)

In [152]:
doacoes_2018['VR_RECEITA'] = doacoes_2018['VR_RECEITA'].str.replace(',','.')
doacoes_2018['VR_RECEITA'] = doacoes_2018['VR_RECEITA'].astype('float64') 

In [153]:
doacoes_2018.columns

Index(['DT_RECEITA', 'NR_CPF_CANDIDATO', 'SQ_CANDIDATO', 'NM_CANDIDATO',
       'DS_FONTE_RECEITA', 'DS_NATUREZA_RECEITA', 'NR_CPF_CNPJ_DOADOR',
       'NM_DOADOR_RFB', 'DS_ESFERA_PARTIDARIA_DOADOR', 'NM_PARTIDO_DOADOR',
       'SQ_CANDIDATO_DOADOR', 'DS_CARGO_CANDIDATO_DOADOR', 'SG_PARTIDO_DOADOR',
       'DS_RECEITA', 'VR_RECEITA'],
      dtype='object')

In [154]:
def limpeza5(linha):
    documento = str(linha["NR_CPF_CNPJ_DOADOR"])
    if len(documento) > 11:
        return documento[:8]
    else:
        return documento

doacoes_2018['doador_raiz'] = doacoes_2018.apply(limpeza5, axis=1)

### Trabalhistas diretas

In [155]:
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
doacoes_trabalhistas = pd.read_csv("resultados/autos_mte/doacoes_diretas_vindas_pessoas_com_autos_de_mte_2018.csv", **kwargs)

In [156]:
doacoes_trabalhistas['valor'] = doacoes_trabalhistas['valor'].str.replace(',','.')
doacoes_trabalhistas['valor'] = doacoes_trabalhistas['valor'].astype('float64') 

In [157]:
doacoes_trabalhistas.columns

Index(['valor', 'politico_id', 'entidade_id', 'entidade_nome', 'doacoes_tipo',
       'origem_do_dinheiro_no_tse', 'data'],
      dtype='object')

In [158]:
doacoes_trabalhistas.sample(5)

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
134,13.2,61464686815,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Partidario,01/11/2018
66,217500.0,03987461411,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,29/08/2018
12,396802.0,19530773587,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,12/09/2018
122,450000.0,00398099863,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,12/09/2018
33,450000.0,13334727149,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,12/09/2018


In [159]:
doacoes_trab = doacoes_trabalhistas.sort_values(by=['politico_id'], ascending=False)

In [160]:
arr = doacoes_trab["politico_id"].unique()
lista = arr.tolist()

In [161]:
len(lista)

70

In [162]:
dados_encontrados = []
for politico in lista:
    valor = 0
    doadores = doacoes_trab[(doacoes_trab['politico_id'] == politico)]
    
    arr = doadores["entidade_id"].unique()
    lista_doadores = arr.tolist()
    
       
    for doador in lista_doadores:
        
        if len(doador) > 11:
            doador = doador[:8]
        
        soma = doacoes_2018[(doacoes_2018['NR_CPF_CANDIDATO'] == politico) & (doacoes_2018['doador_raiz'] == doador)]
        total = soma['VR_RECEITA'].sum()
        valor += total
    
    dicionario = {"politico_id": politico, 
                  "doacao_trabalhista_direta_total": valor}
    dados_encontrados.append(dicionario)    

In [163]:
doacoes_trab_diretas = pd.DataFrame(dados_encontrados)

In [164]:
doacoes_trab_diretas

,politico_id,doacao_trabalhista_direta_total
0,98042734700,584064.05
1,94892393649,900000.00
2,90330862634,900000.00
3,86753991620,394715.10
4,76808742715,900000.00
...,...,...
65,02441369806,900000.00
66,02333314842,900000.00
67,01742564348,100000.00
68,01102125334,42000.00


In [165]:
doacoes_trab_diretas.to_excel('resultados/totais_doacoes/totais_doacoes_diretas_irregularidades_trabalhistas.xlsx',sheet_name='Sheet1',index=False)

In [166]:
busca = doacoes_trabalhistas[(doacoes_trabalhistas['politico_id'] == '02513878442')]
busca['valor'].sum()

1302797.57

In [167]:
busca

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
63,1168128.11,02513878442,29417359000140,CIDADANIA - BRASIL - BR - NACIONAL,direta,Fundo Especial,22/08/2018
64,134669.46,02513878442,29417359000140,CIDADANIA - BRASIL - BR - NACIONAL,direta,Fundo Especial,10/09/2018


In [168]:
busca = doacoes_trabalhistas[(doacoes_trabalhistas['politico_id'] == '03098871946')]
busca['valor'].sum()

893787.19

In [169]:
busca

,valor,politico_id,entidade_id,entidade_nome,doacoes_tipo,origem_do_dinheiro_no_tse,data
79,446893.60,03098871946,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,24/08/2018
80,446893.59,03098871946,00676262000170,PARTIDO DOS TRABALHADORES,direta,Fundo Especial,12/09/2018


In [170]:
busca = doacoes_trabalhistas[(doacoes_trabalhistas['politico_id'] == '07973687793')]
busca['valor'].sum()

1542.5

In [171]:
busca = doacoes_trab_diretas[(doacoes_trab_diretas['politico_id'] == '07973687793')]
busca['doacao_trabalhista_direta_total'].sum()

1542.5

In [172]:
# Versão com os 510 nomes da metodologia
kwargs = {'sep': ',', 'dtype': str, 'encoding': 'utf-8'}
politicos_atual = pd.read_csv("resultados/politicos_2018_versao_12julA.csv", **kwargs)

In [174]:
busca = politicos_atual[(politicos_atual['politico_id'] == '07973687793')]
busca

,nome,foto,nome_urna,uf_eleicao_disputa,politico_id,partido_eleicao,partido_atual,uri,sequencial_candidato,bancada_ruralista,id_interno,doacao_ambiental_direta_total,doacao_ambiental_indireta_total,doacao_ambiental_originaria_total
26,DANIELA MOTE DE SOUZA CARNEIRO,https://www.camara.leg.br/internet/deputado/ba...,Daniela do Waguinho,RJ,07973687793,MDB,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/depu...,190000607467,Sim,204459,NaN,NaN,NaN


In [175]:
# Une coluna totais doacoes ambientais diretas

In [176]:
politicos_atual = pd.merge(politicos_atual, doacoes_trab_diretas, left_on='politico_id', right_on='politico_id', how = 'left')

In [177]:
politicos_atual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 510 entries, 0 to 509
Data columns (total 15 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   nome                               510 non-null    object 
 1   foto                               510 non-null    object 
 2   nome_urna                          510 non-null    object 
 3   uf_eleicao_disputa                 510 non-null    object 
 4   politico_id                        510 non-null    object 
 5   partido_eleicao                    510 non-null    object 
 6   partido_atual                      510 non-null    object 
 7   uri                                510 non-null    object 
 8   sequencial_candidato               510 non-null    object 
 9   bancada_ruralista                  236 non-null    object 
 10  id_interno                         510 non-null    object 
 11  doacao_ambiental_direta_total      101 non-null    object 

In [179]:
busca = politicos_atual[(politicos_atual['politico_id'] == '07973687793')]
busca

,nome,foto,nome_urna,uf_eleicao_disputa,politico_id,partido_eleicao,partido_atual,uri,sequencial_candidato,bancada_ruralista,id_interno,doacao_ambiental_direta_total,doacao_ambiental_indireta_total,doacao_ambiental_originaria_total,doacao_trabalhista_direta_total
26,DANIELA MOTE DE SOUZA CARNEIRO,https://www.camara.leg.br/internet/deputado/ba...,Daniela do Waguinho,RJ,07973687793,MDB,UNIÃO,https://dadosabertos.camara.leg.br/api/v2/depu...,190000607467,Sim,204459,NaN,NaN,NaN,1542.5


In [180]:
politicos_atual.to_csv('resultados/politicos_2018_versao_12julB.csv',index=False)